In [3]:
import os

In [4]:
%pwd

'/Users/shashwatkushwaha/Desktop/ML/TextSummarizer/research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'/Users/shashwatkushwaha/Desktop/ML/TextSummarizer'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_steps: int
    eval_strategy: int
    save_steps: float
    gradient_accumulation_steps: int
    per_device_eval_batch_size: int


In [8]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.params = read_yaml(params_filepath)
        self.config = read_yaml(config_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) ->ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path= config.data_path,
            model_ckpt= config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            eval_strategy = params.eval_strategy,
            eval_steps=params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps,
            per_device_eval_batch_size=params.per_device_eval_batch_size
        )

        return model_trainer_config

In [10]:
from transformers import (
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
    AutoModelForSeq2SeqLM,
    AutoTokenizer
)
from datasets import load_dataset, load_from_disk
import torch

/Users/shashwatkushwaha/Desktop/ML/TextSummarizer/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from textSummarizer.logging import logger

In [12]:
class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        # device = "cuda" if torch.cuda.is_available() else "cpu"
        model_path = os.path.join(self.config.root_dir, 'pegasus-samsum-model')
        tokenizer_path = os.path.join(self.config.root_dir, 'tokenizer')
        if os.path.exists(model_path) and os.path.exists(tokenizer_path):
            logger.info(f"✅ Found existing model at {model_path}. Skipping training!")
            return
        device = "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model = model_pegasus)

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir = self.config.root_dir,
            per_device_train_batch_size = self.config.per_device_train_batch_size,
            per_device_eval_batch_size = self.config.per_device_eval_batch_size,
            num_train_epochs = self.config.num_train_epochs,
            warmup_steps = self.config.warmup_steps,
            weight_decay = self.config.weight_decay,
            logging_steps = self.config.logging_steps,
            eval_strategy = self.config.eval_strategy,
            eval_steps = self.config.eval_steps,
            save_steps = 1e6,
            gradient_accumulation_steps = self.config.gradient_accumulation_steps,
            use_cpu = True
        )

        trainer = Trainer(
            model = model_pegasus,args = trainer_args,
            tokenizer = tokenizer, data_collator = seq2seq_data_collator,
            train_dataset = dataset_samsum_pt['test'],
            eval_dataset=dataset_samsum_pt['validation']
        )

        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,'pegasus-samsum-model'))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,'tokenizer'))

In [13]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-12-23 22:48:12,313 : INFO : common : yaml file : <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2025-12-23 22:48:12,325 : INFO : common : yaml file : <_io.TextIOWrapper name='config/config.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2025-12-23 22:48:12,327 : INFO : common : Created Directory at artifacts]
[2025-12-23 22:48:12,328 : INFO : common : Created Directory at artifacts/model_trainer]
[2025-12-23 22:48:12,329 : INFO : 302213249 : ✅ Found existing model at artifacts/model_trainer/pegasus-samsum-model. Skipping training!]


In [7]:
%pwd

'/Users/shashwatkushwaha/Desktop/ML/TextSummarizer/research'

In [8]:
os.chdir("../")

NameError: name 'os' is not defined

In [5]:
%pwd

'/Users/shashwatkushwaha/Desktop/ML/TextSummarizer'

In [ ]:
if torch.backends.mps.is_available():
        torch.mps.empty_cache()

: 